* [Data Details](https://www.iris.edu/hq/sis/insight#DMC)
* [IRIS API](https://service.iris.edu/fdsnws/dataselect/1/)

In [4]:
%config IPCompleter.greedy=True

import csv, json, os, sys
import requests
from time import sleep

In [13]:
# download data availability
def download_data_availability(save_path='../data/mars/'):
    payload = {
        'option': 'com_ajax',
        'data': 'ELYSE',
        'format': 'json',
        'module': 'seis_data_available'
    }
    r = requests.post('https://www.seis-insight.eu/en/science/seis-data/seis-data-availability', payload)
    with open(os.path.join(save_path, 'data_availability.json'), 'wb') as f:
        f.write(r.content)


In [14]:
download_data_availability()

In [3]:
def convertMetadataTextToCSV():
    metadata = []
    headers = ['network','station','location','channel','quality','sample_rate','start','end']
    with open('data/metadata.txt', 'r') as f:
        for raw_line in f.readlines():
            line = list(filter(lambda x: x != '', raw_line.split(' ')))
            line = list(map(lambda x: x.strip('\n'), line))
            metadata.append(line)

    del metadata[0]

    with open('data/data_params.csv', 'wt') as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(headers) # write header

        for row in metadata:
            csv_writer.writerow(row)

In [4]:
def downloadCSV(line_offset=0, station_filter=None, location_filter=None):
    base = 'http://service.iris.edu/fdsnws/dataselect/1/query'
    with open('data/data_params.csv', 'r') as f:
        for _ in range(line_offset):
            next(f)
        file_counter = line_offset+1
        for line in f.readlines():
            line = line.strip('\n').split(',')
            
            if station_filter is not None:
                if line[1] not in station_filter:
                    continue
            if location_filter is not None:
                if line[2] not in location_filter:
                    continue

            print(line)

            params = {
                'net': line[0],
                'sta': line[1],
                'loc': line[2],
                'cha': line[3],
                'quality': line[4],
                'start': line[6],
                'end': line[7],
                'format': 'geocsv'
            }
            r = requests.get(base, params=params)
            file_name = '_'.join([line[1], line[2], line[3], line[6]])
            with open('data/csv/' + file_name + '.csv', 'wb') as c:
                c.write(r.content)
            
            sleep(1)
            file_counter += 1

In [5]:
convertMetadataTextToCSV()

FileNotFoundError: [Errno 2] No such file or directory: 'data/metadata.txt'

In [6]:
downloadCSV(
    line_offset=0,
    station_filter=['ELYSE'],
    location_filter=['00','05']
)

['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-10T14:16:51.679000Z', '2019-02-10T14:20:53.829000Z']


FileNotFoundError: [Errno 2] No such file or directory: 'data/csv/ELYSE_00_HHU_2019-02-10T14:16:51.679000Z.csv'

In [7]:
from datetime import datetime
print(datetime.now())

2020-03-10 10:13:00.452158


In [8]:
def downloadMiniseed(line_offset=0, station_filter=None, location_filter=None, path='../data/'):
    base = 'http://service.iris.edu/fdsnws/dataselect/1/query'
    with open('data/data_params.csv', 'r') as f:
        for _ in range(line_offset):
            next(f)
        file_counter = line_offset+1
        for line in f.readlines():
            line = line.strip('\n').split(',')
            
            if station_filter is not None:
                if line[1] not in station_filter:
                    continue
            if location_filter is not None:
                if line[2] not in location_filter:
                    continue

            print(line)

            params = {
                'net': line[0],
                'sta': line[1],
                'loc': line[2],
                'cha': line[3],
                'quality': line[4],
                'start': line[6],
                'end': line[7],
                'format': 'miniseed'
            }
            r = requests.get(base, params=params)
            file_name = '_'.join([line[1], line[2], line[3], line[6]])
            with open('../data/miniseed/' + file_name + '.mseed', 'wb') as c:
                c.write(r.content)
            
            sleep(1)
            file_counter += 1

In [10]:
downloadMiniseed(
    line_offset=0,
    station_filter=['ELYSE'],
    location_filter=['00','05']
)

['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-10T14:16:51.679000Z', '2019-02-10T14:20:53.829000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-10T14:21:59.679000Z', '2019-02-10T19:29:09.791000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-10T19:30:15.641000Z', '2019-02-11T00:00:04.619000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-11T00:33:22.630000Z', '2019-02-11T00:37:13.780000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-11T00:38:30.630000Z', '2019-02-11T00:42:32.781000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-11T18:45:14.543000Z', '2019-02-11T18:55:37.693000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-11T21:35:50.533000Z', '2019-02-11T21:46:17.685000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-14T23:46:40.145000Z', '2019-02-14T23:48:14.304000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-15T00:41:21.137000Z', '2019-02-15T00:43:00.296000Z']
['XB', 'ELYSE', '00', 'HHU', 'M', '100.0', '2019-02-16T

In [11]:
downloadMiniseed(
    line_offset=0,
    station_filter=['ELYSE'],
    location_filter=['02']
)

down

['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-09T14:02:51.625000Z', '2019-02-10T14:10:48.020000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-10T14:11:34.480000Z', '2019-02-10T15:17:57.972000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-10T15:18:44.472000Z', '2019-02-12T16:47:27.734000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-14T09:29:25.035000Z', '2019-02-18T07:11:18.012000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-18T09:31:08.510000Z', '2019-02-22T09:49:40.479000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-22T10:24:01.982000Z', '2019-02-23T01:07:25.418000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-23T01:36:40.914000Z', '2019-02-23T05:19:49.394000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-23T05:55:15.892000Z', '2019-02-24T02:34:54.323000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-02-24T04:59:13.798000Z', '2019-03-01T04:03:52.750000Z']
['XB', 'ELYSE', '02', 'MHU', 'M', '2.0', '2019-03-01T08:17:33.222000Z', '

In [3]:
from obspy import read
st = read('data/miniseed/ELYSE_00_HHU_2019-02-10T14:16:51.679000Z.mseed')

In [4]:
st

1 Trace(s) in Stream:
XB.ELYSE.00.HHU | 2019-02-10T14:16:51.679000Z - 2019-02-10T14:20:53.829000Z | 100.0 Hz, 24216 samples

In [5]:
st[0].stats

         network: XB
         station: ELYSE
        location: 00
         channel: HHU
       starttime: 2019-02-10T14:16:51.679000Z
         endtime: 2019-02-10T14:20:53.829000Z
   sampling_rate: 100.0
           delta: 0.01
            npts: 24216
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'M', 'number_of_records': 127, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 65024})

In [6]:
# Download Station Data

# http://ws.ipgp.fr/fdsnws/station/1/query?network=XB&station=ELYSE&level=response&format=xml
base = 'http://ws.ipgp.fr/fdsnws/station/1/query'
params = {
    'network': 'XB',
    'station': 'ELYSE',
    'level': 'response',
    'format': 'xml'
}
r = requests.get(base, params=params)
with open('data/' + 'ELYSE_station' + '.xml', 'wb') as c:
    c.write(r.content)

In [7]:
# Download Catalog

# Standard BED format catalogue
# https://www.seis-insight.eu/static/mqs-catalogs/events_extended_preferredorigin_2019-10-01.xml
!wget https://www.seis-insight.eu/static/mqs-catalogs/events_extended_preferredorigin_2019-10-01.xml -O data/events_extended_preferredorigin_2019-10-01.xml
    
# Mars-specific extension catalogue  
# https://www.seis-insight.eu/static/mqs-catalogs/events_extended_mars_preferredorigin_2019-10-01.xml
!wget https://www.seis-insight.eu/static/mqs-catalogs/events_extended_mars_preferredorigin_2019-10-01.xml -O data/events_extended_mars_preferredorigin_2019-10-01.xml
    
    

--2020-03-10 10:54:56--  https://www.seis-insight.eu/static/mqs-catalogs/events_extended_preferredorigin_2019-10-01.xml
Resolving www.seis-insight.eu (www.seis-insight.eu)... 194.254.225.59
Connecting to www.seis-insight.eu (www.seis-insight.eu)|194.254.225.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1852670 (1.8M) [application/xml]
Saving to: ‘data/events_extended_preferredorigin_2019-10-01.xml’

data/events_extende 100%[===================>]   1.77M   897KB/s    in 2.0s    

2020-03-10 10:54:58 (897 KB/s) - ‘data/events_extended_preferredorigin_2019-10-01.xml’ saved [1852670/1852670]

--2020-03-10 10:54:58--  https://www.seis-insight.eu/static/mqs-catalogs/events_extended_mars_preferredorigin_2019-10-01.xml
Resolving www.seis-insight.eu (www.seis-insight.eu)... 194.254.225.59
Connecting to www.seis-insight.eu (www.seis-insight.eu)|194.254.225.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2243429 (2.1M) [application/xml]
